In [1]:
import glob
import os
import re

from PIL import Image
import torch
from torchvision import transforms
import warnings
warnings.filterwarnings('ignore')

Refer for code: https://pytorch.org/hub/pytorch_vision_resnet/

In [2]:
# Download ImageNet Labels
# !wget https://raw.githubusercontent.com/pytorch/hub/master/imagenet_classes.txt

In [2]:
# Read ImageNet categories
with open('./imagenet_classes.txt', 'r') as f:
    categories = [s.strip() for s in f.readlines()]

print(*categories, sep = '\n')

tench
goldfish
great white shark
tiger shark
hammerhead
electric ray
stingray
cock
hen
ostrich
brambling
goldfinch
house finch
junco
indigo bunting
robin
bulbul
jay
magpie
chickadee
water ouzel
kite
bald eagle
vulture
great grey owl
European fire salamander
common newt
eft
spotted salamander
axolotl
bullfrog
tree frog
tailed frog
loggerhead
leatherback turtle
mud turtle
terrapin
box turtle
banded gecko
common iguana
American chameleon
whiptail
agama
frilled lizard
alligator lizard
Gila monster
green lizard
African chameleon
Komodo dragon
African crocodile
American alligator
triceratops
thunder snake
ringneck snake
hognose snake
green snake
king snake
garter snake
water snake
vine snake
night snake
boa constrictor
rock python
Indian cobra
green mamba
sea snake
horned viper
diamondback
sidewinder
trilobite
harvestman
scorpion
black and gold garden spider
barn spider
garden spider
black widow
tarantula
wolf spider
tick
centipede
black grouse
ptarmigan
ruffed grouse
prairie chicken
peacock

In [3]:
# Path to all image files (jpg/jpeg)
data_path = './data/*.j*'

# Creating a mapping in the form of filename: path
basename_path_mapping = {}

for file in glob.glob(data_path):
    basename_path_mapping[os.path.basename(file).split('.')[0]] = file
    
#print(basename_path_mapping)

In [4]:
# Initializing the VGG model
model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet18', pretrained=True)
model.eval()

Using cache found in C:\Users\thara/.cache\torch\hub\pytorch_vision_v0.10.0
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to C:\Users\thara/.cache\torch\hub\checkpoints\resnet18-f37072fd.pth


  0%|          | 0.00/44.7M [00:00<?, ?B/s]

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [5]:
# Traversing each image and getting top 15 probabilities using VGG16
for label, path in basename_path_mapping.items():
    input_image = Image.open(path)
    preprocess = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])
    input_tensor = preprocess(input_image)
    input_batch = input_tensor.unsqueeze(0) # create a mini-batch as expected by the model

    # move the input and model to GPU for speed if available
    if torch.cuda.is_available():
        input_batch = input_batch.to('cuda')
        model.to('cuda')

    with torch.no_grad():
        output = model(input_batch)
    # Tensor of shape 1000, with confidence scores over Imagenet's 1000 classes
    # print(output[0])
    # The output has unnormalized scores. To get probabilities, you can run a softmax on it.
    probabilities = torch.nn.functional.softmax(output[0], dim=0)
    # print(probabilities)
    print('-----------------')
    print(label)
    print('-----------------')
    top15_prob, top15_catid = torch.topk(probabilities, 15)
    for i in range(top15_prob.size(0)):
        print(categories[top15_catid[i]], round(top15_prob[i].item(),5))
    print()
    print()

-----------------
aquariumfish1
-----------------
goldfish 0.65761
ladybug 0.27407
eft 0.02031
common newt 0.01177
leaf beetle 0.00726
leafhopper 0.00719
puffer 0.00501
hummingbird 0.00139
eel 0.00133
rock beauty 0.00083
European fire salamander 0.00081
admiral 0.00078
goldfinch 0.00074
lorikeet 0.00073
tree frog 0.00071


-----------------
aquariumfish2
-----------------
anemone fish 0.98827
sea anemone 0.01061
coral reef 0.00108
sea slug 1e-05
lionfish 0.0
rock beauty 0.0
coral fungus 0.0
goldfish 0.0
sea urchin 0.0
brain coral 0.0
ping-pong ball 0.0
daisy 0.0
pill bottle 0.0
wooden spoon 0.0
strainer 0.0


-----------------
aquariumfish3
-----------------
impala 0.11058
brambling 0.06933
gazelle 0.06066
tiger cat 0.05347
tiger 0.04527
bittern 0.04262
jay 0.03258
monarch 0.03123
ringlet 0.02754
robin 0.02479
snail 0.02078
quail 0.01997
bee 0.01915
partridge 0.01896
badger 0.0172


-----------------
aquariumfish4
-----------------
sea cucumber 0.42684
sea slug 0.19317
sea anemone 0.10

In [6]:
# Create a mapping of class items to super class
classes = {
    'dinosaur': [51],
    'couch': [831],
    'mushroom': [947, 992, 993, 994, 995, 996, 997],
    'crab': [118, 119, 120, 121, 125],
    'aquariumfish': [1, 5, 107, 123, 124, 327, 395, 396, 397],
    'rose': []
}

# Displaying the names of the classes in imagenet
for key, value in classes.items():
    print('{}: {}'.format(key, [categories[idx] for idx in value]))

dinosaur: ['triceratops']
couch: ['studio couch']
mushroom: ['mushroom', 'agaric', 'gyromitra', 'stinkhorn', 'earthstar', 'hen-of-the-woods', 'bolete']
crab: ['Dungeness crab', 'rock crab', 'fiddler crab', 'king crab', 'hermit crab']
aquariumfish: ['goldfish', 'electric ray', 'jellyfish', 'spiny lobster', 'crayfish', 'starfish', 'gar', 'lionfish', 'puffer']
rose: []


In [8]:
# Traversing each image and getting superclass probability
for label, path in basename_path_mapping.items():
    input_image = Image.open(path)
    preprocess = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])
    input_tensor = preprocess(input_image)
    input_batch = input_tensor.unsqueeze(0) # create a mini-batch as expected by the model

    # move the input and model to GPU for speed if available
    if torch.cuda.is_available():
        input_batch = input_batch.to('cuda')
        model.to('cuda')

    with torch.no_grad():
        output = model(input_batch)
    # Tensor of shape 1000, with confidence scores over Imagenet's 1000 classes
    # print(output[0])
    # The output has unnormalized scores. To get probabilities, you can run a softmax on it.
    probabilities = torch.nn.functional.softmax(output[0], dim=0)

    # Getting the class probability for the image
    image_class = re.sub(r'\d', '', label)
    print('{}: {}'.format(label, sum([probabilities[idx] for idx in classes[image_class]])))

aquariumfish1: 0.6628599166870117
aquariumfish2: 7.668045327591244e-06
aquariumfish3: 0.016488337889313698
aquariumfish4: 0.12555266916751862
aquariumfish5: 0.26336532831192017
aquariumfish6: 0.999971866607666
aquariumfish7: 0.00750817870721221
aquariumfish8: 0.014547625556588173
couch1: 0.9953173398971558
couch2: 0.9947506785392761
couch3: 0.9385818243026733
couch4: 0.747458815574646
couch5: 0.19551992416381836
couch6: 0.0003803329018410295
couch7: 0.8715585470199585
couch8: 0.8166321516036987
crab1: 0.9999720454216003
crab2: 0.998376727104187
crab3: 0.9658325910568237
crab4: 0.7479879856109619
crab5: 0.9998125433921814
crab6: 0.9999982714653015
crab7: 0.005496880039572716
crab8: 0.9316630959510803
dinosaur1: 0.2423488199710846
dinosaur2: 0.3242192268371582
dinosaur3: 0.5918704867362976
dinosaur4: 0.0017637789715081453
dinosaur5: 0.15242500603199005
dinosaur6: 0.0036407471634447575
dinosaur7: 0.992009162902832
dinosaur8: 0.034380361437797546
mushroom1: 0.9984908103942871
mushroom2: 0.